In [ ]:
%pip install -U pip
%pip install -q numpy torchcodec torchaudio ffmpeg matplotlib
%pip list |awk '/numpy|torch|ffmpeg|matplotlib/ {print $1,$2}'

In [ ]:
try:
    import torchaudio as ta
    import torchaudio.transforms as T
    from torch import Tensor, arange
    from torchcodec.decoders import AudioDecoder
    from torchcodec.encoders import AudioEncoder
    import matplotlib.pyplot as plt
    import numpy as np
    from pprint import pprint
except ImportError as e:
    print(f"Import error: {e}")

In [ ]:
# constants and configurations
AUDIO_FILE = "1.MP3"
TARGET_SAMPLING_RATE = 16_000
TARGET_NUM_CHANNEL = 1
TARGET_FORMAT="mp3"
FREQUENCY_BINS = 512

In [ ]:
# load audio file
audioClip = AudioDecoder(AUDIO_FILE)

# explore results
print(f"Returned Type: {type(audioClip)}. Metadata: {type(audioClip.metadata)}")

In [ ]:
# print loaded file metadata
pprint(audioClip.metadata)
pprint(audioClip.get_all_samples().data.shape)

In [ ]:
# resample a clip
def resample(audioClip: AudioDecoder, target_sample_rate: int = 16_000, target_num_channels: int = 1, target_format: str = "mp3") -> AudioDecoder:
    # resample audio to desired format
    samples, sample_rate = audioClip.get_all_samples().data, audioClip.metadata.sample_rate
    encoder = AudioEncoder(samples, sample_rate=sample_rate)

    # resample
    resampledData: t.Tensor = encoder.to_tensor(format=target_format, num_channels=target_num_channels, sample_rate=target_sample_rate)

    # return new AudioDecoder
    return AudioDecoder(resampledData)

In [ ]:
# verify resampling...
resampledClip = resample(audioClip, target_sample_rate=TARGET_SAMPLING_RATE, target_num_channels=TARGET_NUM_CHANNEL, target_format=TARGET_FORMAT)

# view
pprint(resampledClip.metadata)
pprint(type(resampledClip))

In [ ]:
# plot functions
def waveform(clip: AudioDecoder, title: str = "Waveform Plot"):
    # sample rate & channels
    sr = clip.metadata.sample_rate
    channels = clip.metadata.num_channels

    # get samples
    samples = clip.get_all_samples().data
    nc, ns = samples.shape

    # check
    if nc != channels:
        print(f"Mismatch: reported channels in metadata: {channels} differ from Tensor shape: {samples.shape}")
    
    # time scale (num_samples/sample_rate)
    time_scale = arange(0, ns) / sr
    
    fig, axis = plt.subplots(nc,1)
    fig.tight_layout()
    if nc == 1:
        values = samples[0]
        axis.set_xlim([0,time_scale[-1]])
        axis.plot(time_scale, values)
        axis.set_ylabel("Amplitude")
        axis.set_xlabel("Time (s)")
    else:
        for i in range(nc):
            values = samples[i]
            axis[i].set_xlim([0,time_scale[-1]])
            axis[i].plot(time_scale, values)
            axis[i].set_ylabel("Amplitude")
            axis[i].set_xlabel("Time (s)")

    return (fig, axis)

# calculate audio spectrum
def spectrum(clip: AudioDecoder, num_fft_bins: int = FREQUENCY_BINS, title: str = "Power Spectrum"):
    # get samples
    samples = clip.get_all_samples().data
    sample_rate = clip.metadata.sample_rate
    channels = clip.metadata.num_channels

    # spectrum calculator
    s = T.Spectrogram(n_fft=num_fft_bins, power=2)

    # plot
    fig, axis = plt.subplots(channels, 1)
    if channels == 1:
        # calculate spectrum of audio samples (power over frequency)
        mono_data = samples
        spectrogram = s(mono_data)
    
        # convert signal values to dB 
        samples_dB = T.AmplitudeToDB().forward(spectrogram)

        # plot the spectrogram
        axis.set_ylabel("Freq Bins")
        axis.set_xlabel("Time")
        axis.imshow(samples_dB.squeeze(), origin="lower", aspect="auto", interpolation="nearest")
    else:
        # calculate spectrogram of each channel
        for i in range(channels):
            spectrogram = s(samples[i])
            # convert signal values to dB 
            samples_dB = T.AmplitudeToDB().forward(spectrogram)

            # plot the spectrogram
            axis[i].set_ylabel("Freq Bins")
            axis[i].set_xlabel("Time")
            axis[i].imshow(samples_dB.squeeze(), origin="lower", aspect="auto", interpolation="nearest")

    # return the spectrogram
    return (fig, axis)

In [ ]:
# plot waveforms - original
fig_original, axis_original = waveform(audioClip)
# resampled
fig_resampled, axis_resampled = waveform(resampledClip)

In [ ]:
# Display Spectrum - original
spectrum_original, spectrum_axis_original = spectrum(audioClip, num_fft_bins=1024)

# resampled audio
spectrum_resampled, spectrum_axis_resampled = spectrum(resampledClip, num_fft_bins=1024)